<a href="https://colab.research.google.com/github/C12H22O11-2024/retail_demand_forecast/blob/main/20250812_Course_project_Data_Extraction_Time_series_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Load Data


## Install Required Packages

In [ ]:
!pip install -U gdown # Upgrade/install gdown for downloading files from Google Drive

## Import necessary libraries

In [ ]:
# Data manipulation & analysis
import pandas as pd
import numpy as np

In [ ]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Access Google Drive from Colab
from google.colab import drive
from google.colab import files

In [ ]:
# Download Helper
import gdown

# Configurations

In [ ]:
# Dictionary mapping filenames to their Google Drive file IDs
file_information = {
    "stores.csv": "1heKGso4BXMzi4PMY2A_Csb2m-KiuwRxu",
    "items.csv": "1bTB2mV8WdK97zMgXK36hxSysThQ7yIvG",
    "transactions.csv": "1-2i5naktdTZn-EwEfurCQ-qjK1unAGPc",
    "oil.csv": "15JROHp1gVy9E5P_L_-oUUcx9xmMNQw6l",
    "holidays_events.csv": "1yBky_3tA-oXUPC-2QNtDAKqk9bqPY16E"
}

In [ ]:
train_file_id = "1s4fhVpD1oEnixZSi1WtnexaYJJNjKyOc"

In [ ]:
# Define the item families we want to filter: 'GROCERY I', 'BEVERAGES', 'CLEANING'
item_families = ['GROCERY I', 'BEVERAGES', 'CLEANING']

In [ ]:
# Select data before April'14
max_date = '2014-04-01'

In [ ]:
# Chunk size
chunk_size = 10**6

## Mount Google Drive

In [ ]:
drive.mount('/content/drive')  # Mount drive to access dataset files

Mounted at /content/drive


## Download Smaller CSV Files from Google Drive

In [ ]:
# Download each file from Google Drive using gdown
for output, file_id in file_information.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, output, quiet=False)

## Load Downloaded Smaller CSV Files into DataFrames

In [ ]:
df_stores = pd.read_csv("stores.csv")
df_items = pd.read_csv("items.csv")
df_transactions = pd.read_csv("transactions.csv")
df_oil = pd.read_csv("oil.csv")
df_holidays_events = pd.read_csv("holidays_events.csv")

## Load & Filter train.csv in Chunks


In [ ]:
# Download train.csv file from Google Drive
url = f"https://drive.google.com/uc?id={train_file_id}"

output = "train.csv"  # will be saved in Colab’s temp storage
gdown.download(url, output, quiet=False)

# Filter store numbers for the 'Guayas' state
# Extract the unique store numbers from the 'Guayas' state in the stores dataframe
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()

# Get item numbers that belong to the specified item families
items_ids = df_items[df_items['family'].isin(item_families)]

# Create an empty list to store filtered chunks of data
filtered_chunks = []

# Loop through each chunk of data (for large dataset processing)
for chunk in pd.read_csv(output, chunksize=chunk_size, parse_dates=["date"]):
    # Filter the chunk based on store numbers, item numbers
    # Conditions:
    # - Store numbers should be in 'Guayas' state
    # - Item numbers should belong to the selected item families
    chunk_filtered = chunk[(chunk['store_nbr'].isin(store_ids))]
    chunk_filtered = chunk_filtered[(chunk_filtered['date'] < max_date)]
    chunk_filtered = chunk_filtered.merge(items_ids, on="item_nbr", how="inner")

    # Append the filtered chunk to the list of filtered chunks
    filtered_chunks.append(chunk_filtered)

    # Delete the chunk to free up memory (important for large datasets)
    del chunk

# Combine all filtered chunks into a single DataFrame
df_train = pd.concat(filtered_chunks, ignore_index=True)

# Clean up the memory by deleting the list of filtered chunks
del filtered_chunks

# Initial Data Checks

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.describe()

In [ ]:
df_train.info()

In [ ]:
df_train.duplicated().sum()

# Missing Values Check & Handling

## Check for Missing Values

In [ ]:
print("Missing values in train dataset:")
print(df_train.isna().sum(), '\n')  # Checking missing values in df_train

In [ ]:
print("Missing values in oil dataset:")
print(df_oil.isna().sum(), '\n')  # Checking missing values in df_oil

In [ ]:
print("Missing values in transactions dataset:")
print(df_transactions.isna().sum(), '\n')  # Checking missing values in df_transactions

In [ ]:
print("Missing values in holidays_events dataset:")
print(df_holidays_events.isna().sum(), '\n')  # Checking missing values in df_holidays_events

In [ ]:
print("Missing values in items dataset:")
print(df_items.isna().sum(), '\n')  # Checking missing values in df_items

In [ ]:
print("Missing values in stores dataset:")
print(df_stores.isna().sum(), '\n')  # Checking missing values in df_stores

## Handling of Missing Values

In [ ]:
# Fill missing values in `onpromotion` column in df_train
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)  # Assuming missing means not on promotion

In [ ]:
# Fill missing values in `dcoilwtico` column in df_oil using backward fill (bfill)
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].bfill()  # Using the next valid value to fill missing oil prices

## Verification of Missing Values after Handling

In [ ]:
print("Missing values in train dataset after handling:")
print(df_train.isna().sum(), '\n')

In [ ]:
print("Missing values in oil dataset after handling:")
print(df_oil.isna().sum(), '\n')

# Data Cleaning

## Handle Negative Sales (Returns)

In [ ]:
# Identify negative sales before correction
negative_sales_before = df_train[df_train['unit_sales'] < 0]
print("Negative sales before correction:")
print(negative_sales_before.head(), '\n')


In [ ]:
# Replace negative sales with 0 (treat returns as no sales)
df_train['unit_sales'] = df_train['unit_sales'].apply(lambda x: max(x, 0))

In [ ]:
# Verify that there are no negative sales after correction
negative_sales_after = df_train[df_train['unit_sales'] < 0]
if negative_sales_after.empty:
    print("All negative sales have been corrected successfully.")
else:
    print("Negative sales still present after correction:")
    print(negative_sales_after.head())

In [ ]:
df_train.shape

## Check Unique Stores & Items

In [ ]:
# Print unique stores in filtered data compared to original
print(f"Unique stores in filtered data: {df_train['store_nbr'].nunique()} out of {df_stores['store_nbr'].nunique()} stores in the original dataset.")

In [ ]:
# Print unique items in filtered data compared to original
print(f"Unique items in filtered data: {df_train['item_nbr'].nunique()} out of {df_items['item_nbr'].nunique()} items in the original dataset.")

## Detect & Handle Outliers Using Z-Score

In [ ]:
# Compute mean and standard deviation for each store-item group
mean_sales = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform('mean')
std_sales = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform('std')

In [ ]:
# Avoid division by zero by replacing 0 std with 1 (and fill NaNs with 1)
std_sales = std_sales.replace(0, 1).fillna(1)

In [ ]:
# Calculate Z-score for each sale
df_train['z_score'] = (df_train['unit_sales'] - mean_sales) / std_sales

In [ ]:
# Define the threshold for outliers (Z-score > 5 indicates an extreme outlier)
outliers = df_train[df_train['z_score'] > 5]
print(f"Number of outliers detected (Z-score > 5): {len(outliers)}")

In [ ]:
# Display the first few outliers for inspection
outliers.head()

In [ ]:
df_train = df_train[df_train['z_score'] <= 5]

In [ ]:
df_train.shape

##  Fill missing dates with zero sales

In [ ]:
# Convert 'date' column to datetime format
df_train['date'] = pd.to_datetime(df_train['date'])

# Get the minimum and maximum dates in the dataset to create a full date range
min_date = df_train['date'].min()
max_date = df_train['date'].max()

# Generate a full date range from min_date to max_date (daily frequency)
full_date_range = pd.DataFrame({'date': pd.date_range(min_date, max_date, freq='D')})

# Create a DataFrame with all (store, item, date) combinations by merging store-item pairs with full date range
store_item_combinations = df_train[['store_nbr', 'item_nbr']].drop_duplicates()
all_combinations = store_item_combinations.merge(full_date_range, how='cross')

# Merge the full combinations with the original df_train to fill in missing sales for specific dates
df_filled = all_combinations.merge(df_train, on=['store_nbr', 'item_nbr', 'date'], how='left')

# Fill missing sales values with 0 (for days with no sales)
df_filled['unit_sales'] = df_filled['unit_sales'].fillna(0)

# Increase maximum column width and show all columns
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Widen total display width (characters)
pd.set_option('display.max_colwidth', 50)   # Increase column content width

# Check the first few rows of the filled DataFrame
print(df_filled.head())

In [ ]:
df_filled.head()

In [ ]:
df_filled.shape

In [ ]:
df_filled.describe()

In [ ]:
df_filled.isna().sum()

## Drop unnecessary columns

In [ ]:
df_filled = df_filled.drop(columns=['onpromotion']) # drop the onpromotion column
df_filled = df_filled.drop(columns=['z_score']) # drop the z_score column
df_filled = df_filled.drop(columns=['id']) # drop the id column
df_filled = df_filled.drop(columns=['perishable'])
#df_filled = df_filled.drop(columns=['family'])
df_filled = df_filled.drop(columns=['class'])

## Recheck Data after dropping columns

In [ ]:
df_filled.isna().sum()

In [ ]:
df_filled.info()

In [ ]:
df_filled.head()

# Feature Engineering

## Create New time-based features

In [ ]:
df_filled["year"] = df_filled["date"].dt.year

In [ ]:
df_filled["month"] = df_filled["date"].dt.month

In [ ]:
df_filled["day"] = df_filled["date"].dt.day

In [ ]:
df_filled["day_of_week"] = df_filled["date"].dt.dayofweek

## Create lag features (previous sales)

In [ ]:
df_filled["lag_1"] = df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].shift(1)

In [ ]:
df_filled["lag_7"] = df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].shift(7)

In [ ]:
df_filled["lag_14"] = df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].shift(14)

In [ ]:
df_filled["lag_30"] = df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].shift(14)

## Rolling average of unit sales

In [ ]:
df_filled["rolling_avg_7"] = (df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].transform(lambda x: x.rolling(window=7, min_periods=1).mean()))


In [ ]:
df_filled["rolling_stdv_7"] = (df_filled.groupby(["store_nbr", "item_nbr"])["unit_sales"].transform(lambda x: x.rolling(window=7, min_periods=1).std()))


##Fill missing values using forward fill

In [ ]:
df_filled = df_filled.fillna(method ='ffill')

## Drop rows with NaN values after creating lag features

In [ ]:
df_filled.dropna(inplace=True)

## Check Dataset after Feature Engineering

In [ ]:
df_filled.head()

In [ ]:
df_filled.shape

# Save Final Dataset

In [ ]:
#df_filled.to_csv('final_data_guayas_region_20250812.csv', index=False)
#files.download('final_data_guayas_region_20250812.csv')

In [ ]:
df_filled.to_pickle("/content/drive/MyDrive/retail_kaggle_data/final_data_guayas_region_20250815.pkl")
